In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/suchit/Downloads/Txn_data.csv')

/var/folders/8x/7w20nyqs14g5k74pdn82t6980000gn/T/ipykernel_48370/530219657.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/suchit/Downloads/Txn_data.csv')


In [3]:
df.head()

,Unnamed: 0,Date/Time,From_Account_id,To_Account_id,amount
0,0,2024-06-15 00:00:02,9410161,1191872,1000.0
1,1,2024-06-15 00:00:02,9781224,5590231,10000.0
2,2,2024-06-15 00:00:02,4231628,2677870,140.0
3,3,2024-06-15 00:00:05,1339396,869200,1000.0
4,4,2024-06-15 00:00:07,3531317,9053549,5000.0


In [4]:
df['amount'].min()

np.float64(1.0)

In [5]:
df.isna().sum()

Unnamed: 0         0
Date/Time          0
From_Account_id    0
To_Account_id      0
amount             0
dtype: int64

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df = df.dropna()

In [ ]:
df.info()

In [ ]:
# df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
!pip install pyspark

In [ ]:
!pyspark --packages graphframes:graphframes:0.8.2-spark3.0-s_2.12

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AML Detection") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
    .getOrCreate()

spark_df = spark.createDataFrame(df)

# spark.stop()

In [ ]:
# spark_df.filter(F.col("amount") > 10)
# spark_df.groupBy("From_Account_id", "To_Account_id").count().filter(F.col("count") >= 3)


In [ ]:
from pyspark.sql import functions as F
data = spark_df.dropDuplicates(["Date/Time", "From_Account_id", "To_Account_id", "amount"])

aggregated = data.groupBy("From_Account_id", "To_Account_id").agg(
    F.count("*").alias("Total_Transaction_Count"),
    F.avg("amount").alias("Average_Transaction_Amount"),
    F.min("Date/Time").alias("First_Transaction"),
    F.max("Date/Time").alias("Last_Transaction")
)
aggregated = aggregated.withColumn(
    "Temporal_Span", F.datediff("Last_Transaction", "First_Transaction")
)
filtered = aggregated.filter(
    (F.col("Average_Transaction_Amount") > 100)&
     (F.col("Total_Transaction_Count") >= 3))

filtered = filtered.filter(F.col("From_Account_id") != F.col("To_Account_id"))
filtered.show()

In [ ]:
from graphframes import GraphFrame

In [ ]:
from pyspark.sql import SparkSession
from graphframes import GraphFrame

vertices = data.select(F.col("From_Account_id").alias("id")).union(
    data.select(F.col("To_Account_id").alias("id"))
).distinct()

edges = data.select(
    F.col("From_Account_id").alias("src"),
    F.col("To_Account_id").alias("dst"),
    F.col("amount"),
    F.col("Date/Time")
)
graph = GraphFrame(vertices, edges)

# Circular Transactions: Strongly Connected Components (SCC)

In [ ]:
# SCC algorithm
scc = graph.stronglyConnectedComponents(maxIter=10)
circular_clusters = scc.groupBy("component").count().filter(F.col("count") > 1)
circular_clusters.show()

import networkx as nx
import matplotlib.pyplot as plt

edges_pd = edges.toPandas()
G = nx.from_pandas_edgelist(edges_pd, source="src", target="dst", create_using=nx.DiGraph())

In [ ]:
# Sort clusters by count in descending order and select the top 10
flagged_clusters = circular_clusters.orderBy(F.col("count").desc()).limit(10)
flagged_clusters.show()
flagged_clusters_pd = flagged_clusters.toPandas()
print("Flagged Circular Clusters:")
print(flagged_clusters_pd)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def visualize_circular_transactions(account_id, scc_pd, edges_pd):
    component = scc_pd[scc_pd["id"] == int(account_id)]["component"].values
    if len(component) == 0:
        print(f"No component found for account ID {account_id}.")
        return

    component = component[0]
    component_nodes = scc_pd[scc_pd["component"] == component]["id"].values
    component_edges = edges_pd[
        (edges_pd["src"].isin(component_nodes)) & (edges_pd["dst"].isin(component_nodes))
    ]

    if component_edges.empty:
        print(f"No circular transactions found for account ID {account_id}.")
    else:
        G = nx.from_pandas_edgelist(
            component_edges, source="src", target="dst", create_using=nx.DiGraph()
        )
        plt.figure(figsize=(12, 8))
        nx.draw_networkx(
            G,
            with_labels=True,
            node_size=500,
            node_color="lightblue",
            font_size=10,
            arrowsize=15,
        )
        plt.title(f"Circular Transactions for Component {component} (Account: {account_id})")
        plt.show()
scc_pd = scc.toPandas()
edges_pd = edges.toPandas()

In [ ]:
account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
if account_id.lower() in ['n', 'stop']:
    print("Exiting visualization.")
if not account_id.isdigit():
    print("Invalid input. Please enter a valid numeric account ID.")
visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
if account_id.lower() in ['n', 'stop']:
    print("Exiting visualization.")
if not account_id.isdigit():
    print("Invalid input. Please enter a valid numeric account ID.")
visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
if account_id.lower() in ['n', 'stop']:
    print("Exiting visualization.")
if not account_id.isdigit():
    print("Invalid input. Please enter a valid numeric account ID.")
visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
if account_id.lower() in ['n', 'stop']:
    print("Exiting visualization.")
if not account_id.isdigit():
    print("Invalid input. Please enter a valid numeric account ID.")
visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
if account_id.lower() in ['n', 'stop']:
    print("Exiting visualization.")
if not account_id.isdigit():
    print("Invalid input. Please enter a valid numeric account ID.")
visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
# account_id = input("Enter the account ID to visualize circular transactions (or type 'n' to stop): ")
# if account_id.lower() in ['n', 'stop']:
#     print("Exiting visualization.")
# if not account_id.isdigit():
#     print("Invalid input. Please enter a valid numeric account ID.")
# visualize_circular_transactions(account_id, scc_pd, edges_pd)

In [ ]:
# # Plot the graph
# plt.figure(figsize=(12, 8))
# nx.draw_networkx(G, with_labels=False, node_size=10, arrows=True)
# plt.title("Circular Transactions")
# plt.show()

# BFS backtracking


In [ ]:
seed_account = " 282286"
to_account = "4234415"
# 9819145
bfs = graph.bfs(
    fromExpr=f"id = '{seed_account}'",
    toExpr=f"id ='{to_account}'",
    maxPathLength=10
)

bfs.printSchema()
bfs.show(truncate=False)

if bfs.count() > 0:
    layer_edges = bfs.select(F.col("from.id").alias("src"), F.col("to.id").alias("dst"))
    layer_edges_pd = layer_edges.toPandas()
    G = nx.from_pandas_edgelist(layer_edges_pd, source="src", target="dst", create_using=nx.DiGraph())
    plt.figure(figsize=(12, 8))
    nx.draw_networkx(G, with_labels=True, node_size=100, arrows=True)
    plt.title(f"Layering from Seed: {seed_account}")
    plt.show()
else:
    print("No paths found from the seed account.")

In [ ]:
layer_edges = []
for i in range(9):
    from_col = f"from.id" if i == 0 else f"v{i}.id"
    to_col = f"v{i+1}.id" if i < 5 else "to.id"
    edge_col = f"e{i}"
    if edge_col in bfs.columns:
        edges = bfs.select(F.col(from_col).alias("src"),
                           F.col(to_col).alias("dst"),
                           F.col(f"{edge_col}.amount").alias("amount"),
                           F.col(f"{edge_col}.Date/Time").alias("datetime"))
        layer_edges.append(edges)

# Combine all edges
if layer_edges:
    all_edges = layer_edges[0]
    for edge_df in layer_edges[1:]:
        all_edges = all_edges.union(edge_df)
    layer_edges_pd = all_edges.toPandas()
    G = nx.from_pandas_edgelist(layer_edges_pd, source="src", target="dst", create_using=nx.DiGraph())
    plt.figure(figsize=(14, 10))
    nx.draw_networkx(G, with_labels=True, node_size=300, arrows=True, edge_color="blue")
    plt.title(f"Layered Graph from Seed: {seed_account}")
    plt.show()
else:
    print("No valid edges found in BFS result.")

# Smurfing Detection

In [ ]:
# Calculate in-degree and out-degree
degrees = graph.degrees.withColumnRenamed("id", "account")
in_degrees = graph.inDegrees.withColumnRenamed("id", "account").withColumnRenamed("inDegree", "in_degree")  # Rename 'id' to 'account'
out_degrees = graph.outDegrees.withColumnRenamed("id", "account").withColumnRenamed("outDegree", "out_degree") # Rename 'id' to 'account'

# Merge degrees
degree_analysis = degrees.join(in_degrees, "account", "outer").join(out_degrees, "account", "outer")
degree_analysis = degree_analysis.fillna(0)
smurfing_origins = degree_analysis.filter(F.col("out_degree") > 10)
smurfing_endpoints = degree_analysis.filter(F.col("in_degree") > 10)

smurfing_origins.show()
smurfing_endpoints.show()
degree_pd = degree_analysis.toPandas()
plt.figure(figsize=(12, 8))
plt.scatter(degree_pd["in_degree"], degree_pd["out_degree"], alpha=0.7)
plt.xlabel("In-Degree")
plt.ylabel("Out-Degree")
plt.title("Smurfing Accounts: Degree Analysis")
plt.grid()
plt.show()


In [ ]:
# Sort and display top 10 accounts by in-degree and  out-degree
top_in_degree = degree_analysis.orderBy(F.col("in_degree").desc()).limit(10)
top_out_degree = degree_analysis.orderBy(F.col("out_degree").desc()).limit(10)
print("Top 10 High Total Degree Accounts")

degree_analysis = degree_analysis.withColumn("total_degree", F.col("in_degree") + F.col("out_degree"))
top_total_degree = degree_analysis.orderBy(F.col("total_degree").desc()).limit(10)
top_total_degree.show(truncate=False)

smurfing_origins = degree_analysis.orderBy(F.col("out_degree").desc()).limit(10)
print("Top 10 Smurfing Origins:(High Out-Degree Accounts)")
smurfing_origins.show(truncate=False)

smurfing_endpoints = degree_analysis.orderBy(F.col("in_degree").desc()).limit(10)
print("Top 10 Smurfing Endpoints: (High In-degree Accounts)")
smurfing_endpoints.show(truncate=False)

In [ ]:
# Convert top accounts to Pandas for visualization
top_degree_pd = degree_analysis.orderBy(F.col("total_degree").desc()).limit(10).toPandas()

plt.figure(figsize=(12, 8))
plt.bar(top_degree_pd["account"].astype(str), top_degree_pd["total_degree"], color="skyblue")
plt.xlabel("Account ID")
plt.ylabel("Total Degree")
plt.title("Top 10 Accounts by Total Degree")
plt.xticks(rotation=45)
plt.show()

## PageRank

In [ ]:
# Run PageRank
pagerank = graph.pageRank(resetProbability=0.15, maxIter=15)
pagerank.vertices.orderBy(F.desc("pagerank")).show(10)
pagerank_pd = pagerank.vertices.toPandas()

In [ ]:
# Ensure top 10 records are sorted by PageRank score
top_10 = pagerank_pd.sort_values(by="pagerank", ascending=False).head(10)

plt.figure(figsize=(10, 6))
plt.bar(top_10["id"].astype(str), top_10["pagerank"])
plt.xlabel("Account")
plt.ylabel("PageRank Score")
plt.title("Top 10 Suspicious Accounts by PageRank")
plt.xticks(rotation=45)
plt.show()

## Community Detection

In [ ]:
# Label propagation for community detection
communities = graph.labelPropagation(maxIter=5)
communities.show()

community_pd = communities.toPandas()
plt.figure(figsize=(10, 6))
plt.hist(community_pd["label"], bins=20, alpha=0.7)
plt.xlabel("Community Label")
plt.ylabel("Number of Accounts")
plt.title("Community Detection")
plt.grid()
plt.show()

### Added code

In [ ]:
from pyspark.sql.functions import sum, avg, countDistinct, col, variance, max, min,count

In [ ]:
community_sizes = community_pd.groupby("label").size().reset_index(name="size")
top_communities = community_sizes.sort_values(by="size", ascending=False).head(5)

# Create `top_anomalies` as a list of top community labels
top_anomalies = top_communities["label"].tolist()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

community_membership = communities.toPandas()
top_communities = top_anomalies
top_accounts = community_membership[community_membership['label'].isin(top_communities)]

# Graph object using NetworkX
G = nx.Graph()

# Add nodes and edges to the graph
for _, row in top_accounts.iterrows():
    G.add_node(row['id'], label=row['label'])
edges_pd = edges.toPandas()

for _, row in edges_pd.iterrows():
    if row['src'] in top_accounts['id'].values and row['dst'] in top_accounts['id'].values:
        G.add_edge(row['src'], row['dst'])

try:
    import seaborn as sns
    community_colors = {label: color for label, color in zip(top_communities, sns.color_palette("husl", len(top_communities)))}
except ImportError:
    print("Seaborn not found. Please install it using: !pip install seaborn")
    community_colors = {label: plt.cm.get_cmap("viridis")(i / len(top_communities)) for i, label in enumerate(top_communities)}


# Set up the layout for the plot (positioning the nodes)
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(12, 8))
for community in top_communities:
    nodes_in_community = [node for node, data in G.nodes(data=True) if data['label'] == community]
    nx.draw_networkx_nodes(G, pos, nodelist=nodes_in_community, node_size=50, node_color=[community_colors[community]] * len(nodes_in_community))

nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.title("Cluster Plot for Top 5 Communities")
plt.axis("off")
plt.show()

In [ ]:
community_sizes = community_pd['label'].value_counts()
print(community_sizes[:21])